In [2]:
from pathlib import Path
import pandas as pd

In [3]:
folder_path = Path('data')
zip_files = list(folder_path.glob('*.zip'))
print(zip_files)
zip_file = zip_files[-1].absolute()
print(zip_file)

[WindowsPath('data/2023-04-26_00-54-lenta.zip'), WindowsPath('data/2023-04-27_14-27-lenta.zip'), WindowsPath('data/2023-04-30_13-43-lenta.zip'), WindowsPath('data/2023-05-07_21-14-lenta.zip'), WindowsPath('data/2023-05-13_20-33-lenta.zip'), WindowsPath('data/2023-05-19_23-28-lenta.zip'), WindowsPath('data/2023-06-02_12-55-lenta.zip'), WindowsPath('data/2023-06-09_13-46-lenta.zip'), WindowsPath('data/2023-06-24_16-23-lenta.zip'), WindowsPath('data/2023-07-03_20-13-lenta.zip'), WindowsPath('data/2023-07-15_12-32-lenta.zip')]
f:\works\work_002\data\2023-07-15_12-32-lenta.zip


In [4]:
df = pd.read_csv(zip_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28350 entries, 0 to 28349
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Категория      28350 non-null  object 
 1   Название       28350 non-null  object 
 2   Страна         28233 non-null  object 
 3   Скидка         15235 non-null  object 
 4   Цена           28350 non-null  float64
 5   Цена по карте  28338 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.3+ MB


In [10]:
mask = df['Название'].str.lower().str.contains(r'(?=молоко)(?=.* пастериз)(?=.*без змж)', regex=True)
find = df[mask].copy()
mask2 = find['Категория'] == 'Бакалея'
mask3 = find['Название'].str.lower().str.contains('лента')
# test = kilka[mask2].sort_values(by='Цена').copy()
test = find.copy()
test['volume'] = test['Страна'].str.extract('(\d+)').astype(int)
test['Пр. цена'] = round((test['Цена по карте'] / (test['volume'] / 1000)), 2)
mask4 = test['volume']>600
test_min_mean = test[mask4]['Пр. цена'].nsmallest(5).mean()
test_median = test[mask4]['Пр. цена'].median()
test_max_mean = test[mask4]['Пр. цена'].nlargest(5).mean()
print(f'минимум \t{test_min_mean:.2f}')
print(f'медиана \t{test_median:.2f}')
print(f'максимум \t{test_max_mean:.2f}')
print(f'количество позиций: {test.shape[0]}')
display(test.nsmallest(10, columns='Пр. цена'))

минимум 	60.35
медиана 	80.67
максимум 	126.61
количество позиций: 40


,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
2364,"Молоко, сыр, яйцо","Молоко пастеризованное 365 ДНЕЙ 2,5%, без змж,...","Россия, 800 мл",NaN,38.99,36.99,800,46.24
2824,"Молоко, сыр, яйцо","Молоко пастеризованное DANKE 2,5%, без змж, 900мл","Беларусь, 900 мл",NaN,55.79,52.99,900,58.88
1382,"Молоко, сыр, яйцо","Молоко пастеризованное ЭКОНИВА 2,5%, без змж, ...","Россия, 1000 мл",-27%,89.49,64.99,1000,64.99
1537,"Молоко, сыр, яйцо","Молоко пастеризованное НАШЕ ДЕЛО цельное 3,3–6...","Россия, 1000 мл",-31%,94.79,64.99,1000,64.99
1617,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕНТА 2,5%, без змж, 900мл","Россия, 900 мл",-26%,81.09,59.99,900,66.66
1364,"Молоко, сыр, яйцо","Молоко пастеризованное СЧАСТЛИВОЕ ДЕТСТВО 1,5%...","Россия, 900 мл",NaN,64.79,61.49,900,68.32
1535,"Молоко, сыр, яйцо","Молоко пастеризованное BONVIDA 2,5%, без змж, ...","Россия, 1400 мл",-23%,126.39,96.99,1400,69.28
1333,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 3,2%, бе...","Россия, 900 г",-15%,74.69,62.99,900,69.99
1280,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 3,2%, бе...","Россия, 1400 г",-20%,125.69,99.99,1400,71.42
1744,"Молоко, сыр, яйцо","Молоко пастеризованное БЕЖИН ЛУГ 2,5%, без змж...","Россия, 1400 г",-26%,136.89,99.99,1400,71.42


In [11]:
display(test[test['volume']>600].nlargest(5, columns='Пр. цена'))

,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
2296,"Молоко, сыр, яйцо",Молоко пастеризованное PARMALAT Comfort безлак...,"Россия, 900 мл",NaN,164.29,155.99,900,173.32
2083,"Молоко, сыр, яйцо",Молоко пастеризованное PARMALAT Comfort безлак...,"Россия, 900 мл",NaN,147.39,139.99,900,155.54
2706,"Молоко, сыр, яйцо","Молоко пастеризованное АСЕНЬЕВСКАЯ ФЕРМА 3,4–6...","Россия, 900 мл",NaN,107.29,101.89,900,113.21
1324,"Молоко, сыр, яйцо",Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО отборное...,"Россия, 900 г",-15%,101.99,85.99,900,95.54
1648,"Молоко, сыр, яйцо",Молоко пастеризованное СЧАСТЛИВОЕ ДЕТСТВО Отбо...,"Россия, 900 мл",NaN,90.49,85.89,900,95.43


In [59]:
choc_mask_01 = df['Название'].str.lower().str.contains(r'(?=.*\bсоус)(?=.*\bсыр)', regex=True)
# choc_mask_02 = df['Категория'] == 'Кондитерские изделия'
chocolate = df[choc_mask_01].copy().reset_index(drop=True)
# chocolate['volume'] = chocolate['Страна'].str.extract('(\d+)').astype(int)
# chocolate['Пр. цена'] = round((chocolate['Цена по карте'] / (chocolate['volume'] / 100)), 2)
# chocolate[chocolate.columns[1:]].sort_values('Пр. цена')
chocolate

,Категория,Название,Страна,Скидка,Цена,Цена по карте
0,Продукция собственного производства,"Соус майонезный АСТОРИЯ Сырный, 25г","Россия, 25 г",NaN,16.89,15.99
1,"Молоко, сыр, яйцо",Сыр плавленый КАРАТ Для супа и соуса с зеленью...,"Россия, 90 г",NaN,52.69,49.99
2,"Молоко, сыр, яйцо",Сыр плавленый КАРАТ Для супа и соуса с лососем...,"Россия, 90 г",NaN,57.89,54.99
3,Бакалея,Сухарики ржано-пшеничные КИРИЕШКИ Семга с сыро...,"Россия, 85 г",-17%,47.39,38.99
4,Бакалея,"Соус HEINZ Сырный, на основе растительных масе...","Россия, 200 г",-36%,157.89,99.89
5,Бакалея,"Соус CALVE Сырный, на основе растительных масе...","Россия, 230 г",-32%,88.99,59.89
6,Бакалея,"Соус СЛОБОДА Сырный 60%, 220мл","Россия, 220 мл",NaN,78.89,74.89
7,Бакалея,"Соус майонезный АСТОРИЯ Сырный 42%, 233г","Россия, 233 г",NaN,94.79,89.99
8,Бакалея,Смесь MAGGI На второе для макарон в сливочно-с...,"Россия, 30 г",-48%,69.49,35.89
9,Бакалея,"Соус майонезный АСТОРИЯ Сырный 42%, 384г","Россия, 384 г",-20%,147.39,117.89


In [7]:
import requests
from bs4 import BeautifulSoup as BS
import lxml
from pprint import pprint

url = 'https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D1%82%D1%80%D0%B5%D0%B1%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F_%D0%BA%D0%BE%D1%80%D0%B7%D0%B8%D0%BD%D0%B0'

response = requests.get(url)
soup = BS(response.content, 'lxml')

table = soup.find('table', class_='wikitable')
rows = list(table.find_all('tr'))[2:]

sum_list = []
for row in rows[:]:
    out_list = []
    for index, element in enumerate(row.find_all('td')):
        if index > 1:
            per_year = float(element.get_text(strip=True).replace(',', '.'))
            per_month = per_year / 12
            out_list.append(round(per_month, 2))
        else:
            out_list.append(element.get_text(strip=True))
    sum_list.append(out_list)

table_df = pd.DataFrame(sum_list, columns=['name', 'unit', 'works', 'pensioners', 'children'])
table_df

,name,unit,works,pensioners,children
0,Хлебные продукты (хлебимакаронные изделияв пер...,кг,10.54,8.18,6.47
1,Картофель,кг,8.37,6.67,7.34
2,Овощиибахчевые,кг,9.55,8.17,9.38
3,Фруктысвежие,кг,5.00,3.75,9.84
4,Сахарикондитерские изделияв пересчете на сахар,кг,1.98,1.77,1.82
5,Мясопродукты,кг,4.88,4.50,3.67
6,Рыбопродукты,кг,1.54,1.33,1.55
7,Молокои молокопродукты в пересчете на молоко,кг,24.17,21.48,30.06
8,Яйца,штука,17.50,16.67,16.75
9,"Масло растительное,маргарини другиежиры",кг,0.92,0.83,0.42


In [8]:
mask = df['Название'].str.lower().str.contains(r'(?=.*стиральный.*)(?=.*порошок.*)', regex=True)
df_test = pd.DataFrame(df[mask].reset_index(drop='index'))
# mask2 = df_test['Название'].str.lower().str.contains(r'(?=.*)', regex=True)
df_test['Вес'] = df_test['Страна'].apply(lambda x: ''.join(x.split(',', maxsplit=1)[1:]))
df_test['Вес'] = df_test['Вес'].str.extract('(\d+\,?\d*)')
df_test['Вес'] = df_test['Вес'].str.replace(',', '.')
df_test['Вес'] = df_test['Вес'].astype(float).apply(lambda x: x if x > 100 else x * 1000).astype(int)

df_test['Страна'] = df_test['Страна'].apply(lambda x: x.split(',')[0])
# print(df_test['Вес'].unique())
df_test['За кг'] = df_test['Цена по карте'] / df_test['Вес'] * 1000
df_test['За кг'] = df_test['За кг'].round().astype(int)
# df_test[df_test['Страна'] == 'Дания']
df_test[~(df_test['Страна'] == 'Россия')].nsmallest(n=20, columns='За кг')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer